# NASNet-iMaterialist
## TFNW

Train NASNet on the iMaterialist dataset.

@alkari

### Check GPU/CPU memory capacity

In [0]:
# To reset your vm, useful if you can't login to Google Drive
# !kill -9 -1

In [0]:
%%capture
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install -q gputil
!pip install -q psutil
!pip install -q humanize


In [1]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print('Gen RAM Free: ' + humanize.naturalsize( psutil.virtual_memory().available ), ' I Proc size: ' + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.9 GB  I Proc size: 129.0 MB
GPU RAM Free: 11438MB | Used: 1MB | Util   0% | Total 11439MB


### Access GoogleDrive Store

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

### Create folders structure

In [0]:
import os
os.chdir('/content/drive')
os.makedirs('nasnet', exist_ok=True)
os.chdir('nasnet')
os.makedirs('weights', exist_ok=True)

In [0]:
!pwd
!ls ../iMaterialist

### Download model definition

In [0]:
if not os.path.isfile('nasnet.py'):
  !wget https://raw.githubusercontent.com/alkari/Keras-NASNet/master/nasnet.py
if not os.path.isfile('cutout.py'):
  !wget https://raw.githubusercontent.com/alkari/Keras-NASNet/master/cutout.py

### Imports

In [4]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger
from keras.optimizers import Adam
from nasnet import NASNet, NASNetLarge, preprocess_input
from cutout import cutout
import numpy as np
import h5py


Using TensorFlow backend.


### Parameters

In [0]:
weights_file = 'weights/NASNet-iMaterialist.h5'
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.5), cooldown=0, patience=5, min_lr=0.5e-5)
csv_logger = CSVLogger('NASNet-iMaterialist.csv')
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_predictions_acc', save_best_only=True,
                                   save_weights_only=True, mode='max')

In [0]:
batch_size = 16
nb_classes = 129
nb_epoch = 20 
data_augmentation = True

# input image dimensions
img_rows, img_cols = 224, 224
img_channels = 3

### Load Data

In [0]:
def load_dataset(train_file, test_file):
    with h5py.File(train_file, 'r') as hf:
        train_images = hf['train']['images'][:]
        train_labels= hf['train']['labels'][:]

    with h5py.File(test_file, 'r') as hf:
        test_images = hf['test']['images'][:]
        test_labels= hf['test']['labels'][:]
        
    train_labels = np.reshape(train_labels, (len(train_labels), 1))
    test_labels = np.reshape(test_labels, (len(test_labels), 1))
    
    return (train_images,train_labels), (test_images, test_labels)

(X_train, y_train), (X_test, y_test) = load_dataset('../iMaterialist/train_58.h5','../iMaterialist/test_2.h5')

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(1000, 224, 224, 3)
(1000, 1)
(1000, 224, 224, 3)
(1000, 1)


In [0]:
# Convert class vectors to binary class matrices (one-hot-encoding)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


In [10]:
Y_train.shape

(1000, 129)

In [0]:
# preprocess input
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [12]:
# Check memory
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13029        2043        7013         220        3971       10522
Swap:             0           0           0


## Build the model using the auxilary branch to correctly train NASNet


In [0]:
model = NASNetLarge((img_rows, img_cols, img_channels),dropout=0.5, use_auxiliary_branch=True, include_top=True, weights=None, classes=nb_classes)
#model.summary()

In [0]:
optimizer = Adam(lr=1e-3, clipnorm=5)
model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'],
              optimizer=optimizer, metrics=['accuracy'], loss_weights=[1.0, 0.4])

## Begin Training

In [15]:
if not data_augmentation:
    print('Not using data augmentation.')
    
    model.fit(X_train, [Y_train, Y_train],
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_data=(X_test, [Y_test, Y_test]),
              shuffle=True,
              verbose=1,
              callbacks=[lr_reducer, csv_logger, model_checkpoint])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        preprocessing_function=cutout)  # randomly apply cutout

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # wrap the ImageDataGenerator to yield two label batches [y, y] for each input batch X
    def image_generator(image_datagenerator, batch_size):
        iterator = datagen.flow(X_train, Y_train, batch_size=batch_size)

        while True:
            X, y = next(iterator)  # get the next batch
            yield X, [y, y]  # duplicate the labels for each batch

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(image_generator(datagen, batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        validation_data=(X_test, [Y_test, Y_test]),
                        epochs=nb_epoch, verbose=1,
                        callbacks=[lr_reducer, csv_logger, model_checkpoint])

scores = model.evaluate(X_test, [Y_test, Y_test], batch_size=batch_size)
for score, metric_name in zip(scores, model.metrics_names):
    print("%s : %0.4f" % (metric_name, score))
    print('\n')

Using real-time data augmentation.
Epoch 1/20
62/62 [==============================] - 224s 4s/step - loss: 9.7265 - predictions_loss: 6.1116 - aux_predictions_loss: 5.5626 - predictions_acc: 0.0111 - aux_predictions_acc: 0.0141 - val_loss: 23.7820 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 15.9730 - val_predictions_acc: 0.0050 - val_aux_predictions_acc: 0.0090
Epoch 2/20
62/62 [==============================] - 162s 3s/step - loss: 9.1930 - predictions_loss: 5.7993 - aux_predictions_loss: 5.3234 - predictions_acc: 0.0111 - aux_predictions_acc: 0.0081 - val_loss: 23.5090 - val_predictions_loss: 15.9730 - val_aux_predictions_loss: 15.8847 - val_predictions_acc: 0.0090 - val_aux_predictions_acc: 0.0090
Epoch 3/20
53/62 [========================>.....] - ETA: 18s - loss: 8.8907 - predictions_loss: 5.7245 - aux_predictions_loss: 5.1471 - predictions_acc: 0.0118 - aux_predictions_acc: 0.0142

62/62 [==============================] - 162s 3s/step - loss: 8.8963 - predictions_loss: 5.7250 - aux_predictions_loss: 5.1848 - predictions_acc: 0.0121 - aux_predictions_acc: 0.0131 - val_loss: 23.5099 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 16.1020 - val_predictions_acc: 0.0050 - val_aux_predictions_acc: 1.0000e-03
Epoch 4/20
62/62 [==============================] - 161s 3s/step - loss: 8.7252 - predictions_loss: 5.7207 - aux_predictions_loss: 5.0444 - predictions_acc: 0.0071 - aux_predictions_acc: 0.0161 - val_loss: 23.3841 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 16.0214 - val_predictions_acc: 0.0050 - val_aux_predictions_acc: 0.0060
Epoch 5/20
62/62 [==============================] - 161s 3s/step - loss: 8.6818 - predictions_loss: 5.6513 - aux_predictions_loss: 5.3385 - predictions_acc: 0.0121 - aux_predictions_acc: 0.0091 - val_loss: 23.1961 - val_predictions_loss: 15.9569 - val_aux_predictions_loss: 15.9408 - val_predictions_acc: 0.0100 - v

62/62 [==============================] - 162s 3s/step - loss: 8.5408 - predictions_loss: 5.6496 - aux_predictions_loss: 5.1616 - predictions_acc: 0.0111 - aux_predictions_acc: 0.0101 - val_loss: 23.1162 - val_predictions_loss: 15.9892 - val_aux_predictions_loss: 15.8763 - val_predictions_acc: 0.0080 - val_aux_predictions_acc: 0.0150
Epoch 7/20
62/62 [==============================] - 161s 3s/step - loss: 8.3548 - predictions_loss: 5.5491 - aux_predictions_loss: 5.1592 - predictions_acc: 0.0171 - aux_predictions_acc: 0.0141 - val_loss: 23.0848 - val_predictions_loss: 15.9730 - val_aux_predictions_loss: 16.0214 - val_predictions_acc: 0.0090 - val_aux_predictions_acc: 0.0060
Epoch 8/20
62/62 [==============================] - 161s 3s/step - loss: 8.2736 - predictions_loss: 5.5561 - aux_predictions_loss: 5.1293 - predictions_acc: 0.0111 - aux_predictions_acc: 0.0091 - val_loss: 23.0270 - val_predictions_loss: 15.9730 - val_aux_predictions_loss: 16.0536 - val_predictions_acc: 0.0090 - val_a

62/62 [==============================] - 161s 3s/step - loss: 8.3550 - predictions_loss: 5.7546 - aux_predictions_loss: 4.9638 - predictions_acc: 0.0192 - aux_predictions_acc: 0.0171 - val_loss: 23.0186 - val_predictions_loss: 15.9730 - val_aux_predictions_loss: 16.0053 - val_predictions_acc: 0.0090 - val_aux_predictions_acc: 0.0070
Epoch 10/20
62/62 [==============================] - 161s 3s/step - loss: 8.3113 - predictions_loss: 5.6076 - aux_predictions_loss: 5.0096 - predictions_acc: 0.0121 - aux_predictions_acc: 0.0131 - val_loss: 23.1206 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 15.9408 - val_predictions_acc: 0.0050 - val_aux_predictions_acc: 0.0110
Epoch 11/20
62/62 [==============================] - 160s 3s/step - loss: 8.4654 - predictions_loss: 5.7498 - aux_predictions_loss: 5.0683 - predictions_acc: 0.0091 - aux_predictions_acc: 0.0131 - val_loss: 23.1305 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 16.0214 - val_predictions_acc: 0.0050 - val

Epoch 12/20
62/62 [==============================] - 160s 3s/step - loss: 8.2471 - predictions_loss: 5.5734 - aux_predictions_loss: 5.0058 - predictions_acc: 0.0141 - aux_predictions_acc: 0.0161 - val_loss: 23.0705 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 15.9086 - val_predictions_acc: 0.0050 - val_aux_predictions_acc: 0.0130
Epoch 13/20
62/62 [==============================] - 160s 3s/step - loss: 8.3735 - predictions_loss: 5.6728 - aux_predictions_loss: 5.0531 - predictions_acc: 0.0081 - aux_predictions_acc: 0.0071 - val_loss: 22.9570 - val_predictions_loss: 15.9408 - val_aux_predictions_loss: 15.8763 - val_predictions_acc: 0.0110 - val_aux_predictions_acc: 0.0150
Epoch 14/20
60/62 [============================>.] - ETA: 4s - loss: 8.2087 - predictions_loss: 5.5648 - aux_predictions_loss: 5.0221 - predictions_acc: 0.0125 - aux_predictions_acc: 0.0177

62/62 [==============================] - 162s 3s/step - loss: 8.2178 - predictions_loss: 5.5735 - aux_predictions_loss: 5.0254 - predictions_acc: 0.0131 - aux_predictions_acc: 0.0171 - val_loss: 23.0478 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 16.0053 - val_predictions_acc: 0.0050 - val_aux_predictions_acc: 0.0070
Epoch 15/20
62/62 [==============================] - 161s 3s/step - loss: 8.1509 - predictions_loss: 5.5765 - aux_predictions_loss: 4.9721 - predictions_acc: 0.0131 - aux_predictions_acc: 0.0091 - val_loss: 22.9934 - val_predictions_loss: 16.0053 - val_aux_predictions_loss: 16.0536 - val_predictions_acc: 0.0070 - val_aux_predictions_acc: 0.0040
Epoch 16/20
62/62 [==============================] - 160s 3s/step - loss: 8.0868 - predictions_loss: 5.5571 - aux_predictions_loss: 4.8992 - predictions_acc: 0.0202 - aux_predictions_acc: 0.0262 - val_loss: 22.9892 - val_predictions_loss: 16.0053 - val_aux_predictions_loss: 16.0053 - val_predictions_acc: 0.0070 - val

62/62 [==============================] - 160s 3s/step - loss: 8.0174 - predictions_loss: 5.5053 - aux_predictions_loss: 4.8695 - predictions_acc: 0.0282 - aux_predictions_acc: 0.0292 - val_loss: 23.0020 - val_predictions_loss: 16.0375 - val_aux_predictions_loss: 16.0536 - val_predictions_acc: 0.0050 - val_aux_predictions_acc: 0.0040
Epoch 18/20
62/62 [==============================] - 161s 3s/step - loss: 7.9198 - predictions_loss: 5.4550 - aux_predictions_loss: 4.8410 - predictions_acc: 0.0192 - aux_predictions_acc: 0.0333 - val_loss: 22.9743 - val_predictions_loss: 16.0536 - val_aux_predictions_loss: 16.0037 - val_predictions_acc: 0.0040 - val_aux_predictions_acc: 0.0060
Epoch 19/20
62/62 [==============================] - 160s 3s/step - loss: 8.3111 - predictions_loss: 5.8039 - aux_predictions_loss: 4.9138 - predictions_acc: 0.0121 - aux_predictions_acc: 0.0121 - val_loss: 22.9371 - val_predictions_loss: 15.9730 - val_aux_predictions_loss: 15.9569 - val_predictions_acc: 0.0090 - val

1000/1000 [==============================] - 34s 34ms/step
loss : 22.9982


predictions_loss : 15.9730


aux_predictions_loss : 16.0859


predictions_acc : 0.0090


aux_predictions_acc : 0.0020






---














# Backup

In [0]:
# Get filelist
import os
workdir = "/content/drive/iMaterialist"
filelist = []
for file in os.listdir(workdir):
    if file.startswith("train_"):
        filelist.append(os.path.join(workdir,file))